In [1]:
import Pkg
pkgs = [
    "Eisenia",
    "Random",
    "Revise",
    "BioSequences",
    "Test",
    "LightGraphs",
    "Primes",
    "Statistics",
    "Plots",
    "uCSV",
    "DataFrames",
    "HTTP",
    "CodecZlib",
    "FASTX"
]

for pkg in pkgs
    try
        Pkg.add(pkg)
    catch
#         # tried to install an unregistered local package
    end
    eval(Meta.parse("import $pkg"))
end

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  99.9 %Fetching: [======>                                  ]  12.5 %Fetching: [=======>                                 ]  15.7 %                         ]  35.7 %Fetching: [================>                        ]  37.7 %                   ]  50.7 %Fetching: [=====================>                   ]  52.5 %Fetching: [=============================>           ]  70.1 %Fetching: [=============================>           ]  71.4 %Fetching: [==============================>          ]  73.0 %Fetching: [===============================>         ]  75.4 %Fetching: [===============================>         ]  77.3 %Fetching: [================================>        ]  79.6 %  ]  92.6 %Fetching: [======================================>  ]  93.6 %Fetching: [=======================================> ]  96.8 %

  Resolving package versions...
┌ Info: Precompiling Eisenia [453d265d-8292-4a7b-a57c-dce3f9ae6acd]
└ @ Base loading.jl:1278
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Res

In [2]:
TASK = "graph-decomposition-of-related-genomes"
DATE = "2021-02-14"
DIR = "$(homedir())/$(DATE)-$(TASK)"
if !isdir(DIR)
    mkdir(DIR)
end
cd(DIR)

In [3]:
function get_genome_from_ftp_directory(ftp_path)
    fasta_url = "$(ftp_path)/$(basename(ftp_path))_genomic.fna.gz"
    fasta_buffer = CodecZlib.GzipDecompressorStream(IOBuffer(HTTP.get(fasta_url).body))
    return collect(FASTX.FASTA.Reader(fasta_buffer))
end

get_genome_from_ftp_directory (generic function with 1 method)

In [4]:
db = "genbank"
assembly_summary_url = "https://ftp.ncbi.nlm.nih.gov/genomes/$(db)/assembly_summary_$(db).txt"
assembly_summary_response = HTTP.get(assembly_summary_url)
assembly_summary_response_buffer = IOBuffer(assembly_summary_response.body)
assembly_summary_table = DataFrames.DataFrame(uCSV.read(assembly_summary_response_buffer, delim='\t', header=2, skipmalformed=true)...)

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category
,String,String,String,String,String
1,GCA_000001215.4,PRJNA13812,SAMN02803731,,reference genome
2,GCA_000001405.28,PRJNA31257,,,reference genome
3,GCA_000001515.5,PRJNA13184,SAMN02981217,AACZ00000000.4,na
4,GCA_000001545.3,PRJNA20869,SAMN02981238,ABGA00000000.1,na
5,GCA_000001635.9,PRJNA20689,,,reference genome
6,GCA_000001735.2,PRJNA10719,SAMN03081427,,reference genome
7,GCA_000001765.3,PRJNA10626,SAMN00779672,AADE00000000.2,na
8,GCA_000001895.4,PRJNA10629,SAMN02808228,AABR00000000.7,na
9,GCA_000001905.1,PRJNA12569,SAMN02953622,AAGU00000000.3,representative genome


In [5]:
genome_size_table_url = "ftp://ftp.ncbi.nlm.nih.gov/genomes/ASSEMBLY_REPORTS/species_genome_size.txt.gz"
genome_size_table_response = HTTP.get(genome_size_table_url)
genome_size_table_buffer = CodecZlib.GzipDecompressorStream(IOBuffer(genome_size_table_response.body))
genome_size_table = DataFrames.DataFrame(uCSV.read(genome_size_table_buffer, header=1, delim='\t')...)

,#species_taxid,min_ungapped_length,max_ungapped_length,expected_ungapped_length
,Int64,Int64,Int64,Int64
1,9,318000,956000,636876
2,24,2950000,6050000,4500062
3,34,7405000,11109000,9257167
4,56,5994000,15000000,11989871
5,69,4912000,7369000,6140343
6,114,7212000,10820000,9015893
7,122,6222000,9334000,7777997
8,124,5461000,9315000,7387897
9,139,813000,1786000,1299537


In [6]:
joint_table = DataFrames.innerjoin(assembly_summary_table, genome_size_table, on=["species_taxid" => "#species_taxid"])
sort!(joint_table, "expected_ungapped_length")

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid
,String,String,String,String,String,Int64
1,GCA_002829805.1,,,,na,230604
2,GCA_003972145.1,,,,na,230604
3,GCA_003972165.1,,,,na,230604
4,GCA_003972185.1,,,,na,230604
5,GCA_003972205.1,,,,na,230604
6,GCA_003972225.1,,,,na,230604
7,GCA_003972245.1,,,,na,230604
8,GCA_003972265.1,,,,na,230604
9,GCA_000851485.1,,,,reference genome,154834


In [7]:
show(joint_table, allcols=true)

764249×26 DataFrame
│ Row    │ # assembly_accession │ bioproject │ biosample    │
│        │ String               │ String     │ String       │
├────────┼──────────────────────┼────────────┼──────────────┤
│ 1      │ GCA_002829805.1      │            │              │
│ 2      │ GCA_003972145.1      │            │              │
│ 3      │ GCA_003972165.1      │            │              │
│ 4      │ GCA_003972185.1      │            │              │
│ 5      │ GCA_003972205.1      │            │              │
│ 6      │ GCA_003972225.1      │            │              │
│ 7      │ GCA_003972245.1      │            │              │
│ 8      │ GCA_003972265.1      │            │              │
│ 9      │ GCA_000851485.1      │            │              │
│ 10     │ GCA_000856565.1      │            │              │
⋮
│ 764239 │ GCA_902810675.1      │ PRJEB35709 │ SAMEA6374024 │
│ 764240 │ GCA_902810685.1      │ PRJEB35709 │ SAMEA6374021 │
│ 764241 │ GCA_903993795.1      │ PRJEB39553 │ S

In [8]:
# 112903 on refseq
# 795 on genbank!

In [9]:
group = first(DataFrames.groupby(joint_table, "species_taxid"))

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid
,String,String,String,String,String,Int64
1,GCA_002829805.1,,,,na,230604
2,GCA_003972145.1,,,,na,230604
3,GCA_003972165.1,,,,na,230604
4,GCA_003972185.1,,,,na,230604
5,GCA_003972205.1,,,,na,230604
6,GCA_003972225.1,,,,na,230604
7,GCA_003972245.1,,,,na,230604
8,GCA_003972265.1,,,,na,230604


In [10]:
# how many groups are there?

In [11]:
show(group, allcols=true)

8×26 SubDataFrame
│ Row │ # assembly_accession │ bioproject │ biosample │ wgs_master │
│     │ String               │ String     │ String    │ String     │
├─────┼──────────────────────┼────────────┼───────────┼────────────┤
│ 1   │ GCA_002829805.1      │            │           │            │
│ 2   │ GCA_003972145.1      │            │           │            │
│ 3   │ GCA_003972165.1      │            │           │            │
│ 4   │ GCA_003972185.1      │            │           │            │
│ 5   │ GCA_003972205.1      │            │           │            │
│ 6   │ GCA_003972225.1      │            │           │            │
│ 7   │ GCA_003972245.1      │            │           │            │
│ 8   │ GCA_003972265.1      │            │           │            │

│ Row │ refseq_category │ taxid  │ species_taxid │ organism_name     │
│     │ String          │ Int64  │ Int64         │ String            │
├─────┼─────────────────┼────────┼───────────────┼───────────────────┤
│ 1   │ n

In [12]:
genomes = reduce(vcat, map(ftp_path -> get_genome_from_ftp_directory(ftp_path), group[!, "ftp_path"]))

8-element Array{FASTX.FASTA.Record,1}:
 FASTX.FASTA.Record:
   identifier: AY247793.1
  description: Macrobrachium rosenbergii XSV virus capsid protein gene, complete cds
     sequence: CCACGTCTAGCTGCTGACGTTAAATGCAGCCGGGTGGTA…
 FASTX.FASTA.Record:
   identifier: DQ174318.1
  description: Macrobrachium rosenbergii XSV virus nucleocapsid protein CP17 (cp17) and CP16 (cp16) genes, complete cds
     sequence: TTGTAAAACAAGCTTCGGCTTGTTTTACACGTCTAGCTG…
 FASTX.FASTA.Record:
   identifier: DQ521573.1
  description: Macrobrachium rosenbergii XSV virus nucleocapsid protein CP17 gene, complete cds
     sequence: CCACGTCTAGCTGCTGACGTTAAATGCAGCCGGGTGGAA…
 FASTX.FASTA.Record:
   identifier: EF159731.1
  description: Macrobrachium rosenbergii XSV virus capsid protein gene, partial cds
     sequence: ATGAATAAGCGCATTAATAATAATCGGAGAACCATGAGA…
 FASTX.FASTA.Record:
   identifier: EU150130.1
  description: Macrobrachium rosenbergii XSV virus isolate M298 capsid protein gene, complete cds
     sequence: TACA

In [13]:
genome_sequences = FASTX.sequence.(genomes)

8-element Array{BioSequences.LongSequence{BioSequences.DNAAlphabet{4}},1}:
 CCACGTCTAGCTGCTGACGTTAAATGCAGCCGGGTGGTA…TTAGGCATTATTCCGGGTGCGTCGATAAATAAAGACCTT
 TTGTAAAACAAGCTTCGGCTTGTTTTACACGTCTAGCTG…ACAATTAGTTCTCATGGTGGATTTACACATTGACGGTTG
 CCACGTCTAGCTGCTGACGTTAAATGCAGCCGGGTGGAA…TTGTGGCAAGGGGAGACGTATTGGTTAGGCATTATTCCG
 ATGAATAAGCGCATTAATAATAATCGGAGAACCATGAGA…CTGCTAAAGATCCAGTTCCAGTGGTTCGTGCTTTGGTAA
 TACACGTCTAGCTGCTGACGTTAAATGCAGCCGGGTGGT…GGTTAGGCATTATTCCGGGTGCGTCGATAAATAAAGACA
 CGCGGAATTTGGTTACACGTCTAGCTGCTGACGTTAAAT…GGTTAGGCATTATTCCGGGTGCGTCGATAAATAAAGACA
 TACACGTCTAGCTGCTGACGTTAAATGCAGCCGGGTGGT…GGTTAGGCATTATTCCGGGTGCGTCGATAAATAAAGACA
 TACACGTCTAGCTGCTGACGTTAAATGCAGCCGGGTGGT…GGTTAGGCATTATTCCGGGTGCGTCGATAAATAAAGACA

In [14]:
graph = Eisenia.KmerGraph(BioSequences.BigDNAMer{7}, genome_sequences)

Eisenia.KmerGraph{BioSequences.BigMer{BioSequences.DNAAlphabet{2},7}}({964, 1057} undirected simple Int64 graph, Dict{LightGraphs.SimpleGraphs.SimpleEdge{Int64},Array{Eisenia.EdgeEvidence,1}}(Edge 263 => 853 => [Eisenia.EdgeEvidence(1, 593), Eisenia.EdgeEvidence(2, 619), Eisenia.EdgeEvidence(3, 593), Eisenia.EdgeEvidence(5, 594), Eisenia.EdgeEvidence(6, 607), Eisenia.EdgeEvidence(7, 594), Eisenia.EdgeEvidence(8, 594)],Edge 87 => 88 => [Eisenia.EdgeEvidence(2, 9)],Edge 17 => 104 => [Eisenia.EdgeEvidence(1, 305), Eisenia.EdgeEvidence(2, 331), Eisenia.EdgeEvidence(3, 305), Eisenia.EdgeEvidence(4, 243), Eisenia.EdgeEvidence(5, 306), Eisenia.EdgeEvidence(6, 319), Eisenia.EdgeEvidence(7, 306), Eisenia.EdgeEvidence(8, 306)],Edge 38 => 473 => [Eisenia.EdgeEvidence(6, 4)],Edge 373 => 600 => [Eisenia.EdgeEvidence(1, 163), Eisenia.EdgeEvidence(2, 189), Eisenia.EdgeEvidence(3, 163), Eisenia.EdgeEvidence(4, 101), Eisenia.EdgeEvidence(5, 164), Eisenia.EdgeEvidence(6, 177), Eisenia.EdgeEvidence(7, 16

In [ ]:
# graph = Eisenia.KmerGraph(BioSequences.BigDNAMer{7}, genome_sequences)

In [ ]:
# 	Comment
# H 	Header
# S 	Segment
# L 	Link
# C 	Containment
# P 	Path

# H	VN:Z:1.0
# S	11	ACCTT
# S	12	TCAAGG
# S	13	CTTGATT
# L	11	+	12	-	4M
# L	12	-	13	+	5M
# L	11	+	13	+	3M
# P	14	11+,12-,13+	4M,5M

In [20]:
edge = first(LightGraphs.edges(graph.graph))

Edge 1 => 8

In [ ]:
orientations = assess_path_orientations(path, kmers, initial_orientation)
if orientations == nothing
    path_likelihood = 0.0
    oriented_path = OrientedKmer[]
else
    oriented_path = orient_path(path, orientations)

In [66]:
function kmer_pair_to_oriented_path(kmer_pair, graph)
    path = [kmer_pair[1], kmer_pair[2]]
    orientations = Eisenia.assess_path_orientations(path, graph.kmers, true)
    if orientations == nothing
        orientations = Eisenia.assess_path_orientations(path, graph.kmers, false)
    end
#     if orientations == nothing
#         path = reverse(path)
#         @show "reversing path..."
#         orientations = Eisenia.assess_path_orientations(path, graph.kmers, true)
#         if orientations == nothing
#             orientations = Eisenia.assess_path_orientations(path, graph.kmers, false)
#         end
#     end
    if orientations == nothing
        @show graph.kmers[path]
        error()
    end
    return Eisenia.orient_path(path, orientations)
end

kmer_pair_to_oriented_path(edge.src => edge.dst, graph)

2-element Array{Eisenia.OrientedKmer,1}:
 Eisenia.OrientedKmer(1, true)
 Eisenia.OrientedKmer(8, true)

In [68]:
GFA_file = "$(DIR)/resolving-related-genomes.k-7.gfa"
open(GFA_file, "w") do io
    println(io, "H\tVN:Z:1.0")
    for (i, kmer) in enumerate(graph.kmers)
        fields = ["S", "$i", string(kmer)]
        line = join(fields, '\t')
        println(io, line)
    end
    for edge in LightGraphs.edges(graph.graph)
        oriented_src, oriented_dst = kmer_pair_to_oriented_path(edge.src => edge.dst, graph)
        overlap = length(graph.kmers[1]) - 1
        link = ["L",
                    oriented_src.index,
                    oriented_src.orientation ? '+' : '-',
                    oriented_dst.index,
                    oriented_dst.orientation ? '+' : '-',
                    "$(overlap)M"]
        line = join(link, '\t')
        println(io, line)
    end
end

In [ ]:
# This worked great!!
# try and run bandage from the command line and show the graph inline
# show with all possible nodes merged and default